In [ ]:
# Dependencies...

# Mount the Drive.
from google.colab import drive
drive.mount('/content/drive')


!pip install scipy==1.1.0
!pip install tensorflow==1.15.0

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data Preparation...

import scipy.misc
import random
import numpy as np

train_batch_pointer = 0
val_batch_pointer = 0
xs = []
ys = []
accels = []

dataPath = "/content/drive/My Drive/Behavorial-Clonng-Self-driving-cars/indian_dataset/"
fileNamePrefix = "circuit2_x264.mp4 "

with open(dataPath+"data.txt", "r") as f:
  for line in f:
    xs.append(dataPath + fileNamePrefix + str(int(line.split()[0])).zfill(5)+".jpg")
    ys.append(float(line.split()[1])* scipy.pi / 180)
    accels.append(float(line.split()[2])* scipy.pi / 180)

print("Done")
num_images = len(xs)

train_xs = xs[:int(len(xs) * 0.8)]
train_ys = ys[:int(len(xs) * 0.8)]
train_accels = accels[:int(len(xs) * 0.8)]

val_xs = xs[-int(len(xs) * 0.2):]
val_ys = ys[-int(len(xs) * 0.2):]
val_accels = accels[-int(len(xs) * 0.2):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)


def LoadTrainBatch(batch_size, steer):
  global train_batch_pointer
  x_out = []
  y_out = []
  accels_out = []
  for i in range(0, batch_size):
    if (steer):
      x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66,200]) / 255.0)
    else:
      x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [112, 112]) / 255.0)
    y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    accels_out.append([train_accels[(train_batch_pointer + i) % num_train_images]]) 
  train_batch_pointer += batch_size
  return x_out, y_out, accels_out


def LoadValBatch(batch_size, steer):
  global val_batch_pointer
  x_out = []
  y_out = []
  accels_out = []
  for i in range(0, batch_size):
    if (steer):
      x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66,200]) / 255.0)
    else:
      x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [112, 112]) / 255.0)
    y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    accels_out.append([val_accels[(val_batch_pointer + i) % num_val_images]])
  val_batch_pointer += batch_size
  return x_out, y_out, accels_out

Done


In [ ]:
# Steering Angle Model...

import tensorflow as tf

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

# isTheta = tf.placeholder(tf.bool)
x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

# if (tf.cond(tf.eisTheta = True)):

# if (tf.cond(tf.equal(isTheta, tf.constant(True)), lambda: True, lambda: False)):
# if (isTheta is not None):
y = tf.multiply(tf.atan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output
# else:
# 	y = tf.matmul(h_fc4_drop, W_fc5) + b_fc5 #linear output

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# Steering Angle Model Training...

import os
from tensorflow.core.protobuf import saver_pb2

LOGDIR = '/content/drive/My Drive/Behavorial-Clonng-Self-driving-cars/Weight_Saves'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

logs_path = '/content/drive/My Drive/Behavorial-Clonng-Self-driving-cars/logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 200

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    print("Training Epoch " + str(epoch) + ", Step " + str(i))
    xss, yss, accelss = LoadTrainBatch(batch_size, True)
    train_step.run(feed_dict={x: xss, y_: yss, keep_prob: 0.8})
    if i % 10 == 0:
      xss, yss, accelss = LoadValBatch(batch_size, True)
      loss_value = loss.eval(feed_dict={x:xss, y_: yss, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xss, y_: yss, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      checkpoint_path = os.path.join(LOGDIR, "model_steer_new.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Training Epoch 0, Step 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Streaming output truncated to the last 5000 lines.
Training Epoch 0, Step 86
Training Epoch 0, Step 87
Training Epoch 0, Step 88
Training Epoch 0, Step 89
Training Epoch 0, Step 90
Epoch: 0, Step: 90, Loss: 5.62011
Training Epoch 0, Step 91
Training Epoch 0, Step 92
Training Epoch 0, Step 93
Training Epoch 0, Step 94
Training Epoch 0, Step 95
Training Epoch 0, Step 96
Training Epoch 0, Step 97
Training Epoch 0, Step 98
Training Epoch 0, Step 99
Training Epoch 0, Step 100
Epoch: 0, Step: 100, Loss: 5.55165
Training Epoch 0, Step 101
Training Epoch 0, Step 102
Training Epoch 0, Step 103
Training Epoch 0, Step 104
Training Epoch 0, Step 105
Training Epoch 0, Step 106
Training Epoch 0, Step 107
Training Epoch 0, Step 108
Training Epoch 0, Step 109
Training Epoch 0, Step 110
Epoch: 0, Step: 110, Loss: 5.50454
Training Epoch 0, Step 111
Training Epoch 0, Step 112
Training Epoch 0, Step 113
Training Epoch 0, Step 114
Training Epoch 0, Step 115
Training Epoch 0, Step 116
Training Epoch 0, Step

In [ ]:
# Acceleration Model...

# import tensorflow as tf

def weight_variable_accel(shape):
  
  # initializer = tf.contrib.layers.variance_scaling_initializer()
  initializer = tf.contrib.layers.xavier_initializer()
  return tf.Variable(initializer(shape))

  # initial = tf.truncated_normal(shape, stddev=0.1)
  # return tf.Variable(initial)

def bias_variable_accel(shape):

  # initializer = tf.contrib.layers.variance_scaling_initializer()
  initializer = tf.contrib.layers.xavier_initializer()
  return tf.Variable(initializer(shape))


  # initial = tf.constant(0.1, shape=shape)
  # return tf.Variable(initial)


def conv2d_accel(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME')

x_accel = tf.placeholder(tf.float32, shape=[None, 112, 112, 3])
y_accel_ = tf.placeholder(tf.float32, shape=[None, 1])

# isTheta = tf.placeholder(tf.bool)
x_image_accel = x_accel

# Block 1
#first convolutional layer
W_conv1_accel = weight_variable_accel([3, 3, 3, 256])
b_conv1_accel = bias_variable_accel([256])
h_conv1_accel = tf.nn.relu(conv2d_accel(x_image_accel, W_conv1_accel, 1) + b_conv1_accel)

h_conv1_accel_pool = tf.nn.max_pool(h_conv1_accel, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME')


keep_prob_accel_conv = tf.placeholder(tf.float32)
h_conv1_drop_accel = tf.nn.dropout(h_conv1_accel_pool, keep_prob_accel_conv)

#second convolutional layer
W_conv2_accel = weight_variable_accel([3, 3, 256, 128])
b_conv2_accel = bias_variable_accel([128])
h_conv2_accel = tf.nn.relu(conv2d_accel(h_conv1_drop_accel, W_conv2_accel, 1) + b_conv2_accel)

h_conv2_accel_pool = tf.nn.max_pool(h_conv2_accel, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME')
h_conv2_drop_accel = tf.nn.dropout(h_conv2_accel_pool, keep_prob_accel_conv)

print(h_conv2_drop_accel.shape)

#FCL 1
W_fc1_accel = weight_variable_accel([28*28*128, 128])
b_fc1_accel = bias_variable_accel([128])

# print(h_conv4_accel_pool.shape)
h_conv2_flat_accel = tf.reshape(h_conv2_drop_accel, [-1, 28*28*128])
h_fc1_accel = tf.nn.relu(tf.matmul(h_conv2_flat_accel, W_fc1_accel) + b_fc1_accel)

keep_prob_accel = tf.placeholder(tf.float32)
h_fc1_drop_accel = tf.nn.dropout(h_fc1_accel, keep_prob_accel)


#FCL 2
W_fc2_accel = weight_variable_accel([128, 64])
b_fc2_accel = bias_variable_accel([64])

h_fc2_accel = tf.nn.relu(tf.matmul(h_fc1_drop_accel, W_fc2_accel) + b_fc2_accel)
h_fc2_drop_accel = tf.nn.dropout(h_fc2_accel, keep_prob_accel)

#FCL 3
W_fc3_accel = weight_variable_accel([64, 1])
b_fc3_accel = bias_variable_accel([1])

y_accel = tf.matmul(h_fc2_drop_accel, W_fc3_accel) + b_fc3_accel #linear output

(?, 28, 28, 128)


In [ ]:
# Acceleration Model Training...

import cv2
# import os
# from tensorflow.core.protobuf import saver_pb2

# standard step - reset computation graphs
# tf.reset_default_graph()

saveDirectory = '/content/drive/My Drive/Behavorial-Clonng-Self-driving-cars/Weight_Saves'
L2NormConst = 0.001

all_vars = tf.trainable_variables()
# model_accel_vars = [k for k in all_vars if 'accel' in k.name]

loss_accel = tf.reduce_mean(tf.square(tf.subtract(y_accel_, y_accel))) #+ tf.add_n([tf.nn.l2_loss(v) for v in all_vars]) * L2NormConst
train_step_accel = tf.train.AdadeltaOptimizer(1., 0.95, 1e-6).minimize(loss_accel) # These are default parameters to the Adadelta optimizer
# train_step_accel = tf.train.AdamOptimizer().minimize(loss_accel)
sess_accel = tf.Session()
sess_accel.run(tf.global_variables_initializer())
saver_accel = tf.train.Saver(all_vars)

# create a summary to monitor cost tensor
tf.summary.scalar("loss_accel", loss_accel)

# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# op to write logs to Tensorboard
logs_path = '/content/drive/My Drive/Behavorial-Clonng-Self-driving-cars/logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 50
batch_size = 80

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    print("Training Epoch " + str(epoch) + ", Step " + str(i))
    xss, yss, accelss = LoadTrainBatch(batch_size, False)
    train_step_accel.run(feed_dict={x_accel: xss, y_accel_: accelss, keep_prob_accel: 0.5, keep_prob_accel_conv: 0.25}, session = sess_accel)

    if i % 10 == 0:
      xss, yss, accelss = LoadValBatch(batch_size, False)
      loss_value_accel = loss_accel.eval(feed_dict={x_accel:xss, y_accel_: accelss, keep_prob_accel: 1.0, keep_prob_accel_conv: 1.0}, session = sess_accel)
      print("Epoch: %d, Step: %d, Accel Loss: %g " % (epoch, epoch * batch_size + i, loss_value_accel))

    if i % batch_size == 0:
      accel_checkpoint_path = os.path.join(saveDirectory, "model_accel.ckpt")
      filename_accel = saver_accel.save(sess_accel, accel_checkpoint_path)

  print("Model saved in file: %s" % filename_accel)

Training Epoch 0, Step 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Streaming output truncated to the last 5000 lines.
Epoch: 37, Step: 3260, Accel Loss: 0.000402481 
Training Epoch 37, Step 301
Training Epoch 37, Step 302
Training Epoch 37, Step 303
Training Epoch 37, Step 304
Training Epoch 37, Step 305
Training Epoch 37, Step 306
Training Epoch 37, Step 307
Training Epoch 37, Step 308
Training Epoch 37, Step 309
Training Epoch 37, Step 310
Epoch: 37, Step: 3270, Accel Loss: 0.00102503 
Training Epoch 37, Step 311
Training Epoch 37, Step 312
Training Epoch 37, Step 313
Training Epoch 37, Step 314
Training Epoch 37, Step 315
Training Epoch 37, Step 316
Training Epoch 37, Step 317
Training Epoch 37, Step 318
Training Epoch 37, Step 319
Training Epoch 37, Step 320
Epoch: 37, Step: 3280, Accel Loss: 3.45981e-05 
Training Epoch 37, Step 321
Training Epoch 37, Step 322
Training Epoch 37, Step 323
Training Epoch 37, Step 324
Training Epoch 37, Step 325
Training Epoch 37, Step 326
Training Epoch 37, Step 327
Training Epoch 37, Step 328
Training Epoch 37, Ste

In [ ]:
# Testing Dependinces...


from subprocess import call
import math
import numpy as np
import cv2

def distance(x1 , y1 , x2 , y2): 
  return math.sqrt(math.pow(x2 - x1, 2) +
                   math.pow(y2 - y1, 2) * 1.0)

def smoothFactor(predictedValue, previousValue):
  return 0.2 * pow(abs((predictedValue - previousValue)), 2.0 / 3.0) * (predictedValue - previousValue) / abs(predictedValue - previousValue + 1e-5)

feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

def Get_OF_factor(prev_frame, cur_frame, cur_ang, prev_OF):

  previous_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
  present_frame = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)

  p0 = cv2.goodFeaturesToTrack(present_frame, mask = None, **feature_params)
  p1, st, err = cv2.calcOpticalFlowPyrLK(previous_frame, present_frame, p0, None, **lk_params)

  good_new = p1[st==1]
  good_old = p0[st==1]

  distSum = 0
  for i,(new,old) in enumerate(zip(good_new,good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
    dist = distance (a, b, c, d)
    distSum += dist
  
  opticalParam = None
  if prev_OF != None:
    if (good_old.shape[0] < 10):
      opticalParam = prev_OF
    else:
      opticalParam = distSum/(math.pow(good_old.shape[0], 2)+math.pow(cur_ang, 2)+1)*100
  else:
    opticalParam = distSum/(math.pow(good_old.shape[0], 2)+math.pow(cur_ang, 2)+1)*100
  
  return opticalParam

In [ ]:
# Testing...

import tensorflow as tf
import math

# standard step - reset computation graphs
tf.reset_default_graph()

max_gas_value = 5
alpha_accel = 0.1
beta_accel = 0.9
tanh_alpha = 0.18

g_steer = tf.Graph() ## This is one graph
g_accel = tf.Graph() ## This is another graph


with g_steer.as_default():
  # from google.colab import files
  # src = list(files.upload().values())[0]
  # open('model_steer.py','wb').write(src)
  import model_steer
  all_vars = tf.trainable_variables()
  print(len(all_vars))
  sess_steer = tf.Session(graph = g_steer)
  saver_steer = tf.train.Saver(all_vars)
  saver_steer.restore(sess_steer, "/content/drive/My Drive/Behavorial-Clonng-Self-driving-cars/Weight_Saves/model_steer_new.ckpt")


with g_accel.as_default():
  # from google.colab import files
  # src = list(files.upload().values())[0]
  # open('model_accel.py','wb').write(src)
  import model_accel
  all_vars = tf.trainable_variables()
  sess_accel = tf.Session(graph = g_accel)
  saver_accel = tf.train.Saver(all_vars)
  saver_accel.restore(sess_accel, "/content/drive/My Drive/Behavorial-Clonng-Self-driving-cars/Weight_Saves/model_accel.ckpt")

# image pred...
full_image = scipy.misc.imread(xs[1], mode="RGB")    
image = scipy.misc.imresize(full_image[-150:], [112, 112]) / 255.0
image_steer = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0

acceleration = model_accel.y_accel.eval(feed_dict={model_accel.x_accel: [image], model_accel.keep_prob_accel: 1.0, model_accel.keep_prob_accel_conv: 1.0}, session = sess_accel)[0][0] * 180.0 / scipy.pi
degrees = model_steer.y.eval(feed_dict={model_steer.x: [image_steer], model_steer.keep_prob: 1.0}, session = sess_steer)[0][0] * 180.0 / scipy.pi
pred_deg = round(degrees, 2)
# smooth angle with the prev one

optical_adjusted_gas = max_gas_value * math.tanh(tanh_alpha* Get_OF_factor(scipy.misc.imread(xs[0], mode="RGB"), full_image, pred_deg, None))  #OF
accel_integrated = optical_adjusted_gas*alpha_accel + acceleration * beta_accel
# smooth acc with prev one.

print(acceleration, pred_deg)

4.8050940032778 2.9
